<a href="https://colab.research.google.com/github/Strojove-uceni/su2-final-projects-2025-nesnerova/blob/main/SU2_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Závěrečný projekt k předmětu SU2
# Detekce a sledování klatrinem potažených jamek pomocí metod strojového učení

Autor: Sabina Nešněrová


#Abstrakt

Tento projekt se zabývá aplikací metod strojového učení při analýze mikroskopických dat TIRF-SIM s cílem detekovat a sledovat klatrinem potažené jamky (CCP) během endocytózy. Výchozí baseline metody (jednoduchý morfologický detektor, CNN a Nearest-Neighbour tracker) vykazovaly nízkou přesnost, a proto byly implementovány a experimentálně porovnány pokročilejší přístupy.

V projektu byly porovnávány různé architektury neuronových sítí (U-Net, Res-U-Net), různé postupy filtrování výstupu a několik metod trackingu ( Kalmanův filtr, Nearest-Neighbour přístup a knihovna Norfair). Součástí práce bylo rovněž ladění parametrů (threshold, max_dist, sigma).

Přestože se nepodařilo překonat referenční metodu SOTA, dosažené výsledky výrazně překonaly původní baseline a přiblížily se kvalitě SOTA v metrikách HOTA, AssA a DetA. Projekt tak demonstruje, jak lze kombinací pokročilejších segmentačních modelů a sofistikovanějšího trackingu významně zlepšit analýzu dynamiky CCP.


#Metodika
Při buněčné endocytóze dochází k procesu, kdy buňka přijímá molekuly a další menší buňky. Klatrinem potažené jamky tzv. CCP se vytvářejí na povrchu buňky, rostou a následně se oddělí a tím vytvoří tzv. vezikuly, které pak přenáší potřebné živiny do buňky.

Studium tohoto procesu přináší vědcům důležité informace o buněčných přeměnách, příjmu živin a podobně. Problémem však je, jak detekovat a sledovat mikroskopická data. Tuto otázku řeší strojové učení a jeho metody aplikované na mikroskopická data, která jsou upravena tzv. TIRF-SIM metodou, což je metoda kombinující osvětlení pouze tenké vrstvy buňky, díky čemuž se můžeme zaměřit pouze na povrch buňky, a rozlišení nad rámec klasických světelných mikroskoupů. Díky této metodě jsme schopni jasně pozorovat CCP jamky a jejich přeměny.

Primárním cílem projektu je co nejvíce se přiblížit výkonu referenční metody SOTA, případně dosáhnout ještě lepších výsledků. Metoda SOTA je metoda dosahující nejlepších výsledků v úloze mikroskopických dat.

V této úloze byly první vstupní metody Konvoluční neuronová síť a Jednoduchý morfologický detektor spolu se SOTA vyhodnoceny následovně:

| Metrika | SOTA | Původní CNN |Původní Morphological|
|---------|------|-------------|----------------------|
| HOTA    | 0.74 | 0.31        |         0.23         |
| AssA    | 0.67 | 0.27        |             0.23     |
| DetA    | 0.81 | 0.36        |                 0.24 |


Naším cílem je tedy zvýšit obě metriky AssA i DetA, čímž se zvýší i výsledná HOTA. Jak lze vidět z tabulky u původní CNN byl obecně největší problém s AssA metrikou, která se vztahuje k trackingu.

Budou porovnány dvě metody detektoru a dvě metody trackingu, přičemž v závěru bude uvedena nejlepší kombinace spolu s parametry dávající nejbližší čislo HOTA k SOTA metodě.

##U-Net
U-Net je konvoluční neuronová síť navržená pro segmentaci obrazu na úrovni pixelů. Její architektura se skládá z kodéru, který extrahuje rysy, a dekodéru, který je opět převádí na prostorové mapy.
V tomto projektu U-Net odhaduje pravděpodobnostní mapu výskytu CCP, z níž se poté získají lokální maxima jako detekce.

Byla vybrána z těchto důvodů:
- detekuje malé objekty
- Zlepšuje AssA (méně časté přeskakování mezi ID)

Model byl natrénován v souboru [training_unet](https://raw.githubusercontent.com/Strojove-uceni/su2-final-projects-2025-nesnerova/refs/heads/main/training_unet.ipynb) a zde jsou jen staženy výsledky k další interpretaci


##ResUNet
Tato metoda je vylepšenou verzí U-Net díky residuálním blokům převzatým z ResNet.

Byla vybrána z těchto důvodů
- stabilní trénink
- rychlejší konvergence
- schopnost učit se jemnějším detailům

Model byl natrénován v souboru [training_resunet](https://raw.githubusercontent.com/Strojove-uceni/su2-final-projects-2025-nesnerova/refs/heads/main/training_resunet.ipynb) a zde jsou jen staženy výsledky k další interpretaci

##Nearest-Neighbor
Tento přístup byl ponecház z původního baseline souboru pro porovnání s metodami zmíněnými níže. Je založen na přiřazení objektu z dalšího snímku k objektu z předchozího snímku podle vzdálenosti.

##Kalmanův filtr
Tato metoda je určená pro odhady stavů dynamických systémů, které jsou zatíženy šumem, což je právě případ buněčné endocytózy. Funguje na základě dvou kroků - predikce polohy na základě předchozího stavu a následná korekce podle nové detekce.

Byla vybrána z těchto důvodů:
- Odhaduje zašuměné systémy
- přesná detekce poloh a spojení v trajektorie
- zlepšuje DetA

##Hungarian algoritmus
Optimalizační metoda pro přiřazování objektů k sobě. Zde se přiřazují k sobě predikce z Kalmanova filtru a nové detekce snímku

##Knihovna Norfair
Norfair je moderní knihovna pro multi-object tracking, která používá jednoduché aktualizační pravidlo a různé metriky vzdálenosti. Byla použita Manhattanova vzdálenost
V projektu byla testována jako alternativa ke Kalmanovu filtru.

In [ ]:
#@title Instalace potřebných balíčků

!pip install filterpy
!pip install norfair
!pip install gdown

In [ ]:
#@title Propojení s GitHub
!rm -rf project
!git clone https://github.com/Strojove-uceni/su2-final-projects-2025-nesnerova.git project

import sys
sys.path.append("/content/project")

In [ ]:
#@title Import knihoven

from src.models import UNet, ResUNet
import gdown
import os, requests, zipfile
import matplotlib.animation as animation
from IPython.display import HTML, display, clear_output
import subprocess
import matplotlib as mpl
mpl.rcParams['animation.embed_limit'] = 150
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from matplotlib.cm import ScalarMappable
from matplotlib.colors import Normalize
import matplotlib.collections as mc
import pandas as pd
from matplotlib.patches import Rectangle
from matplotlib.animation import FuncAnimation
from torch.utils.data import IterableDataset
from typing import Tuple, Union
from scipy.ndimage import gaussian_filter, maximum_filter
import ipywidgets as widgets
import math
from scipy import spatial, optimize
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as torch_data
import time
import threading
from norfair import Detection, Tracker
import random
from filterpy.kalman import KalmanFilter
from scipy.optimize import linear_sum_assignment
import cv2
from tqdm import tqdm
import seaborn as sns

In [ ]:
#@title Stažení TIRF-SIM dat

def download_and_unzip(url, extract_to, chain_path):
    """Download and unzip using a verified SSL certificate."""
    if os.path.exists(extract_to):
        print(f"The directory '{extract_to}' already exists. Skipping download and extraction.")
        return

    local_zip = os.path.basename(url)
    print(f"📥 Downloading {local_zip} (approx 1min)...")

    try:
        response = requests.get(url, stream=True, verify=chain_path, timeout=20)
        response.raise_for_status()
        with open(local_zip, "wb") as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        print(f"✅ {local_zip} downloaded successfully ({os.path.getsize(local_zip)} bytes)")

        print(f"📂 Extracting to '{extract_to}/'...")
        os.makedirs(extract_to, exist_ok=True)
        with zipfile.ZipFile(local_zip, "r") as zip_ref:
            zip_ref.extractall(extract_to)
        os.remove(local_zip)
        print(f"✅ Extraction completed to '{extract_to}'")

    except requests.exceptions.RequestException as e:
        print(f"❌ Download error: {e}")
    except zipfile.BadZipFile:
        print("❌ The downloaded file is not a valid ZIP archive.")
    except Exception as e:
        print(f"⚠️ Unexpected error: {e}")

chain_path = "/content/chain-harica-cross.pem"
print("🔐 Downloading SSL certificate chain...")
cert_url = "https://pki.cesnet.cz/_media/certs/chain-harica-rsa-ov-crosssigned-root.pem"
r = requests.get(cert_url, timeout=10, stream=True)
r.raise_for_status()
with open(chain_path, "wb") as f:
    f.write(r.content)
print("✅ Certificate chain downloaded.\n")
zip_url = "https://su2.utia.cas.cz/files/labs/final2025/val_and_sota.zip"
extract_directory = "/content/val_data"
download_and_unzip(zip_url, extract_directory, chain_path)

print("\n🎯 All done!")



In [ ]:
#@title Funkce z baseline

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

    # Ensure deterministic behavior in cuDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)


def open_tiff_file(name: str) -> np.ndarray:
    img = Image.open(name)
    frames = []
    for i in range(img.n_frames):
        img.seek(i)
        frames.append(np.array(img))

    return np.array(frames).squeeze()

def loading_html(message: str) -> str:
    return f"""
<div id="loading-msg">
  <br /><br />
  <b><span style='display:inline-block;animation:flipPause 2s ease infinite;'>⏳</span>
  {message}</b>
</div>
<style>
@keyframes flipPause {{
  0% {{transform:rotate(0deg);}}
  40%{{transform:rotate(180deg);}}
  50%{{transform:rotate(180deg);}}
  90%{{transform:rotate(360deg);}}
  100%{{transform:rotate(360deg);}}
}}
</style>
"""

html_code_reconstruction = loading_html("Showing input validation data, please wait...")

def replace_loading_js(message: str, delay_ms: int = 0) -> str:
    return f"""
<script>
  setTimeout(function(){{
    var loadingDiv = document.getElementById("loading-msg");
    if (loadingDiv) {{
      loadingDiv.innerHTML = '<br /><b>{message}</b>';
    }}
  }}, {delay_ms});
</script>
"""

replace_loading_js_default = replace_loading_js("Only the first 50 frames are displayed.")
replace_loading_js_empty = replace_loading_js("")


class OTF:
    """
    The Optical Transfer Function of the optical system.
    Generates a 2-D image from an exponential approximation of the ideal OTF.
    """

    def __init__(self, na: float, wavelength: float, pixel_size: float, image_size: int, curvature: float):
        """
        :param na: Numerical aperture of the optical system.
        :param wavelength: Wavelength of the emitted light in nanometers.
        :param pixel_size: Physical size of the image pixel in micrometers.
        :param image_size: Width and height of the image.
        :param curvature: Bend of the model OTF function in [0, 1] range where 1 is a perfect OTF.
        :param samples: Number of values to sample.
        """
        cutoff_frequency = 1000 * 2 * na / wavelength  # in micrometer^-1 (for incoherent imaging)
        self.image_cutoff = cutoff_frequency * pixel_size * image_size
        self.image_size = image_size
        self.curvature = curvature

    def __call__(self, size: int = None, x_shift: float = 0, y_shift: float = 0) -> np.ndarray:
        """
        Generate a 2-D image representation.
        :param size: Width and height of the generated image.
        :param x_shift: Sub-pixel shift along the x-axis of the OTF center.
        :param y_shift:Sub-pixel shift along the y-axis of the OTF center.
        :return: 2-D representation of the OTF
        """
        if size is None:
            size = self.image_size

        x, y = np.meshgrid(np.hstack([np.arange(size // 2), np.arange(-size // 2, 0)]),
                           np.hstack([np.arange(size // 2), np.arange(-size // 2, 0)]))
        distance_to_origin = np.hypot(x + x_shift, y + y_shift)

        return self.value(np.minimum(distance_to_origin / self.image_cutoff, 1))

    def double(self, x_shift: float = 0, y_shift: float = 0) -> np.ndarray:
        return self(self.image_size * 2, x_shift, y_shift)

    def value(self, x):
        return (2 / np.pi) * (np.arccos(x) - x * np.sqrt(1 - x * x)) * self.curvature ** x

def apodization_filter(dist_ratio: float, bend: float, size: int) -> np.ndarray:
    """
    Generate an apodization filter that can be directly multiplied with the summed fft result image.

    :param dist_ratio: A coefficient that transform the point distance in pixels to a value where [0, 1] range
        corresponds to the extended OTF support.
    :param bend: A coefficient in the [0, 1] range defining how much medium frequencies should be augmented.
    :param size: Size of the apodization filter.
    :return: np.ndarray of size (size, size) of the apodization filter.
    """
    x, y = np.meshgrid(np.hstack([np.arange(size // 2), np.arange(-size // 2, 0)]),
                       np.hstack([np.arange(size // 2), np.arange(-size // 2, 0)]))

    distance = np.hypot(x, y) * dist_ratio
    mask = np.bitwise_and(0 <= distance, distance < 1)

    apo = np.power(
        (2 / np.pi) * (np.arccos(distance, where=mask) - distance * np.sqrt(1 - distance * distance, where=mask)),
        bend,
        where=mask
    )

    return np.where(mask, apo, 0)


def wiener_filter(shifts, otf: OTF, w: float, size: int) -> np.ndarray:
    """
    Generate a wiener filter denominator so that it can be directly multiplied with the summed fft result image.

    :param shifts: A container of length 3 of shifts of the second component for each of the 3 orientations.
    :param otf: OTF object.
    :param w: Wiener parameter.
    :param size: Size of the wiener filter.
    :return: np.ndarray of size (size, size) of the Wiener filter denominator.
    """
    otf0 = np.abs(otf(size)) ** 2
    wiener = 3 * otf0

    for i in range(3):
        otf1 = np.abs(otf(size, shifts[i][1], shifts[i][0])) ** 2
        otf2 = np.abs(otf(size, -shifts[i][1], -shifts[i][0])) ** 2

        wiener += otf1 + otf2

    return 1 / (wiener + w * w)

def map_otf_support(components: np.ndarray, shifts, otf: OTF) -> None:
    """
    Multiply conjugated OTF to corresponding parts of it's support in the separated and shifted components. Values
    outside the OTF support are set to zero. This function alters the original images.

    :param components: np.ndarray of shape (9, size, size) of padded separated and shifted components in frequency space.
    :param shifts: A container of length 3 of shifts of the second component for each of the 3 orientations.
    :param otf: OTF object
    """
    size = components.shape[-1]

    components[::3] *= np.conjugate(otf(size))
    for i in range(3):
        components[i * 3 + 1] *= np.conjugate(otf(size, shifts[i][1], shifts[i][0]))
        components[i * 3 + 2] *= np.conjugate(otf(size, -shifts[i][1], -shifts[i][0]))


def fourier_shift(fft_image: np.ndarray, x_shift: Union[float, np.ndarray], y_shift: Union[float, np.ndarray]) -> np.ndarray:
    """
    Use the fourier shift theorem to perform a sub-pixel shift of a frequency image in the spatial domain.

    It is implemented in a way that allows the x_shift and y_shift parameters to be a np.ndarray of arbitrary shape with
    the last two axes being np.newaxis or None. The returned array shape is then x/y_shift.shape + (height, width)

    :param fft_image: np.ndarray of shape (height, width).
    :param x_shift: Sub-pixel shift along the x-axis.
    :param y_shift: Sub-pixel shift along the y-axis.
    :return: np.ndarray of shape (height, width) of the shifted frequency image.
    """
    height, width = fft_image.shape[-2:]

    spatial_image = np.fft.ifft2(fft_image)

    x_indices = np.arange(-width // 2, width // 2, dtype=int)
    y_indices = np.arange(-height // 2, height // 2, dtype=int)

    x = np.exp(-1j * 2 * np.pi * x_shift * x_indices[None, :] / width)
    y = np.exp(-1j * 2 * np.pi * y_shift * y_indices[:, None] / height)

    shifted_spatial_image = spatial_image * (x * y)

    return np.fft.fft2(shifted_spatial_image)


def shift_components(components: np.ndarray, shifts) -> None:
    """
    Use the fourier shift theorem to perform a sub-pixel shift of frequency components in the spatial domain. This
    function alters the original images.

    :param components: np.ndarray of shape (9, height, width) of padded separated components in frequency space.
    :param shifts: A container of length 3 of shifts of the second component for each of the 3 orientations.
    :return: np.ndarray of shape (9, height, width) of shifted components.
    """
    for i in range(3):
        components[i * 3 + 1, ...] = fourier_shift(components[i * 3 + 1, ...], shifts[i][1], shifts[i][0])
        components[i * 3 + 2, ...] = fourier_shift(components[i * 3 + 2, ...], -shifts[i][1], -shifts[i][0])


def pad_components(components: np.ndarray) -> np.ndarray:
    """
    Pad an image to twice its size from the center, keeping the corners intact.
    This makes it possible to pad images that are NOT centered using np.fft.fftshift.

    :param components: np.ndarray of shape (..., size, size) of separated components in frequency space.
    :return: np.ndarray of shape (..., size, size) padded with zeros from the center.
    """
    size = components.shape[-1]
    padded_components = np.zeros(components.shape[:-2] + (size * 2, size * 2), dtype=np.complex128)

    x, y = np.meshgrid(
        np.hstack([np.arange(size // 2), np.arange(size * 3 // 2, size * 2)]),
        np.hstack([np.arange(size // 2), np.arange(size * 3 // 2, size * 2)])
    )

    padded_components[..., y, x] = components

    return padded_components

def _component_separation_matrix(phase_offset: float = 0, modulation: float = 1) -> np.ndarray:
    phases = np.array([0, 2 * np.pi / 3, 4 * np.pi / 3]) + phase_offset

    M = np.ones((3, 3), dtype=np.complex128)


    M[:, 1] = 0.5 * modulation * (np.cos(phases) + 1j * np.sin(phases))
    M[:, 2] = 0.5 * modulation * (np.cos(-phases) + 1j * np.sin(-phases))

    return np.linalg.inv(M)

def separate_components(fft_images: np.ndarray, phase_offsets=np.zeros(3), phase_modulations=np.ones(3)) -> np.ndarray:
    """
    Compute the spectral separation of components.

    :param fft_images: np.ndarray of shape (9, height, width) of images in frequency space.
    :param phase_offsets: A container of length 3 of phase offsets for each of the 3 orientations, zeros by default.
    :param phase_modulations: A container of length 3 of phase modulations for each of the 3 orientations,
        ones by default.
    :return: np.ndarray of shape (9, height, width) of separated components.
    """
    separation_matrices = np.array([_component_separation_matrix(phase_offsets[i], phase_modulations[i])
                                    for i in range(3)])

    return np.einsum('aij,ajkl->aikl',separation_matrices,
                     fft_images.reshape(3, 3, *fft_images.shape[1:])).reshape(9, *fft_images.shape[1:])


def run_reconstruction(fft_images: np.ndarray, otf: OTF, shifts, phase_offsets, modulations, config: dict) -> Tuple[np.ndarray, np.ndarray]:
    """
    Perform a SIM reconstruction using the provided parameters.

    :param fft_images: np.ndarray of shape (9, height, width) of images in frequency space.
    :param otf: OTF object
    :param shifts: A container of length 3 of shifts of the second component for each of the 3 orientations.
    :param phase_offsets: A container of length 3 of phase offsets for each of the 3 orientations.
    :param modulations: A container of length 3 of phase modulations for each of the 3 orientations.
    :param config: A dictionary of SIM reconstruction parameters. It must contain the following keys: ["wavelength",
        "na", "px_size", "apo_cutoff", "wiener_parameter", "apo_bend"]
    :return: fft_result, spatial_result
    """
    size = fft_images.shape[-1]
    cutoff = 1000 * 2 * config["na"] / config["wavelength"]
    apo_dist_ratio = 1 / (config["px_size"] * config["apo_cutoff"] * cutoff * size)

    components = separate_components(fft_images, phase_offsets, modulations)
    components = pad_components(components)

    shift_components(components, shifts)
    map_otf_support(components, shifts, otf)

    wiener = wiener_filter(shifts, otf, config["wiener_parameter"], size * 2)
    apodization = apodization_filter(apo_dist_ratio, config["apo_bend"], size * 2)

    fft_result = np.sum(components, 0) * wiener * apodization
    spatial_result = np.real(np.fft.ifft2(fft_result))

    return fft_result, spatial_result

def illumination_pattern(angle, frequency, phase_offset, amplitude, size) -> np.ndarray:
    n = size // 2
    Y, X = np.mgrid[-n:n, -n:n]
    ky, kx = np.sin(angle) * frequency, np.cos(angle) * frequency
    return 1 + amplitude * np.cos(2 * np.pi * (X * kx + Y * ky) + phase_offset)


class PerlinNoise:
    """Adapted from: https://github.com/pvigier/perlin-numpy"""

    def __init__(self, size: int, res: int):
        """
        :param size: length of both dimensions of the generated noise image
        :param res: number of periods of noise to generate along each axis
        """

        meshgrid = np.mgrid[0:res:res / size, 0:res:res / size]
        self.grid = np.stack(meshgrid) % 1

        self.t = self._fade(self.grid)
        self.d = size // res
        self.sample_size = (res + 1, res + 1)

    def __call__(self) -> np.array:
        """
        Returned values are always in the [0, 1] range with ~0.5 mean
        """
        angles = 2 * np.pi * np.random.random_sample(self.sample_size)
        gradients = np.dstack((np.cos(angles), np.sin(angles))).repeat(self.d, 0).repeat(self.d, 1)

        n00 = (np.dstack((self.grid[0], self.grid[1])) * gradients[:-self.d, :-self.d]).sum(2)
        n10 = (np.dstack((self.grid[0] - 1, self.grid[1])) * gradients[self.d:, :-self.d]).sum(2)
        n01 = (np.dstack((self.grid[0], self.grid[1] - 1)) * gradients[:-self.d, self.d:]).sum(2)
        n11 = (np.dstack((self.grid[0] - 1, self.grid[1] - 1)) * gradients[self.d:, self.d:]).sum(2)

        n0 = n00 * (1 - self.t[0]) + n10 * self.t[0]
        n1 = n01 * (1 - self.t[0]) + n11 * self.t[0]

        return 0.5 + 2 ** -0.5 * (n0 * (1 - self.t[1]) + n1 * self.t[1])

    @staticmethod
    def _fade(t):
        return 6 * t ** 5 - 15 * t ** 4 + 10 * t ** 3


class SyntheticDataset(IterableDataset):
    def __init__(self, contrast_fg_range: tuple[float,float] = (0.0, 1.0), contrast_bg_range: tuple[float,float] = (0.0, 1.0)):
        self.patch_size = 128

        self.contrast_fg_range = contrast_fg_range
        self.contrast_bg_range = contrast_bg_range

        self.frequency = 0.17
        self.amplitude = 1.0

        self.config = {
            "na": 1.49,
            "wavelength": 512,
            "px_size": 0.07,
            "wiener_parameter": 0.1,
            "apo_cutoff": 2.0,
            "apo_bend": 0.9
        }

        self.otf = OTF(self.config['na'], self.config['wavelength'], self.config['px_size'], self.patch_size // 2, 0.3)
        self.otf_mult = self.otf(self.patch_size)

        self.perlin = PerlinNoise(self.patch_size, 1)

    def _simulate_sim(self, image):
        angle0 = np.random.uniform(0, np.pi * 2)
        phase_offsets = np.random.uniform(0, np.pi * 2, 3)

        shifts = [(self.frequency * self.patch_size * np.sin(angle0 + i * np.pi / 3),
                   self.frequency * self.patch_size * np.cos(angle0 + i * np.pi / 3))
                  for i in range(3)]

        illumination = np.stack([illumination_pattern(angle0 + i // 3 * np.pi / 3,
                                                      self.frequency,
                                                      phase_offsets[i // 3] + (i % 3) * np.pi * 2 / 3,
                                                      self.amplitude,
                                                      self.patch_size)
                                 for i in range(9)])


        fg_c = np.random.uniform(*self.contrast_fg_range)
        bg_c = np.random.uniform(*self.contrast_bg_range)
        foreground = 250 + fg_c * 500
        background = 50 + bg_c * 50

        high_res_image = (image * foreground + background) * self.perlin()

        ix = np.fft.fft2(illumination * high_res_image)
        hix = self.otf_mult * ix
        dhix = hix.reshape(9, 2, self.patch_size // 2, 2, self.patch_size // 2).sum((1, 3)) / 4
        low_res_images = np.random.poisson(np.fft.ifft2(dhix).real).astype(np.float64)

        noisy_shifts = [np.random.triangular((y - 0.25, x - 0.25), (y, x), (y + 0.25, x + 0.25)) for y, x in shifts]
        noisy_phase_offsets = np.random.normal(phase_offsets, np.pi / 6)
        noisy_amplitudes = np.random.normal(self.amplitude, 0.1, 3)

        reconstruction = run_reconstruction(np.fft.fft2(low_res_images), self.otf, noisy_shifts, noisy_phase_offsets, noisy_amplitudes, self.config)[1]
        return (reconstruction - np.mean(reconstruction)) / np.std(reconstruction)

class SyntheticCCPDataset(SyntheticDataset):
    def __init__(self,min_n: int = 5, max_n: int = 15, radius: float = 2.5,
        contrast_fg_range: tuple[float,float] = (0.0, 1.0),
        contrast_bg_range: tuple[float,float] = (0.0, 1.0)):
        super().__init__(
            contrast_fg_range=contrast_fg_range,
            contrast_bg_range=contrast_bg_range
        )

        self.patch_size = 128

        # Possible positions
        yy, xx = np.mgrid[15:self.patch_size - 1:16, 15:self.patch_size - 1:16]
        self.yy = yy.flatten()
        self.xx = xx.flatten()

        # validate
        if not (0 <= min_n < max_n <= 49):
            raise ValueError("Require 0 <= min_n < max_n <= 49")
        self.min_n, self.max_n = min_n, max_n

        self.max_offset = 8
        assert self.max_offset < 16

        # Beta params
        self.beta_a = 2
        self.beta_b = 1

        # CCP shape params
        self.radius = radius
        self.thickness = 1.0

        # Patch positions
        self.yyy, self.xxx = np.mgrid[:self.patch_size, :self.patch_size]

    def __iter__(self):
        while True:
            yield self.data_sample()

    def data_sample(self):
        # Generate positions and classes
        n = np.random.randint(self.min_n, self.max_n)
        indices = np.random.choice(len(self.yy), size=n, replace=False)
        offsets = np.random.uniform(-self.max_offset, self.max_offset, (n, 2))
        positions = np.column_stack([self.yy[indices], self.xx[indices]]) + offsets
        classes = np.random.beta(self.beta_a, self.beta_b, n) * 0.9 + 0.1

        # Generate simulated HR image and output target
        target_distance = classes * self.radius
        distance = np.hypot(self.yyy[..., None] - positions[:, 0], self.xxx[..., None] - positions[:, 1])
        abs_distance = np.abs(distance - target_distance)
        parts = np.where(abs_distance > self.thickness, 0, np.log(np.interp(abs_distance / self.thickness, [0, 1], [np.e, 1])))
        full_image = np.sum(parts, -1)

        distances = np.maximum(classes - distance / ((1 - classes) * 2 + self.radius + self.thickness * 2), 0)
        y = np.minimum(np.sum(distances, -1), 1)

        # Generate simulated SIM image
        x = super()._simulate_sim(full_image)

        return x, y



def show_tracking(data, image_stack,
                  y_min=512, y_max=768, x_min=256, x_max=512,
                  tail_length=10, color='yellow', show_roi=True):
    """
    Visualize CCP trajectories within a defined ROI, from either a CSV file or a DataFrame.

    Args:
        data (str | pd.DataFrame): Path to CSV file or a DataFrame containing trajectories.
        image_stack (np.ndarray): 3D numpy array (frames, height, width).
        y_min, y_max, x_min, x_max (int): ROI bounds.
        tail_length (int): Number of frames for trajectory tails.
        color (str): Trajectory color.
        show_roi (bool): Whether to display cyan ROI rectangle on the full frame.
    """

    if isinstance(data, str):
        trajectories_df = pd.read_csv(data)
    elif isinstance(data, pd.DataFrame):
        trajectories_df = data.copy()
    else:
        raise TypeError("`data` must be a CSV file path or a pandas DataFrame.")

    tracks_in_roi = trajectories_df.groupby('track_id').filter(
        lambda t: (y_min < t.y.mean() < y_max) and (x_min < t.x.mean() < x_max)
    )

    html_code_linking = loading_html("Loading cropped region and tracks, please wait...")
    display(HTML(html_code_linking))

    if show_roi:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(image_stack[0], cmap='magma')
        rect = Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                         linewidth=2, edgecolor='cyan', facecolor='none')
        ax.add_patch(rect)
        ax.set_title("Full image (cyan box shows cropped region)")
        plt.show()

    def animate_trajectories_cropped(trajectories_df, image_stack, tail_length=10, color='yellow'):
        cropped_stack = image_stack[:, y_min:y_max, x_min:x_max]

        fig, ax = plt.subplots()
        im = ax.imshow(cropped_stack[0], cmap='magma')
        particles = trajectories_df['track_id'].unique()

        line_collections = {pid: mc.LineCollection([], linewidths=1, colors=color) for pid in particles}
        for lc in line_collections.values():
            ax.add_collection(lc)

        dot = ax.scatter([], [], s=5, c=color)

        def animate(i):
            im.set_array(cropped_stack[i])

            window = trajectories_df[
                (trajectories_df['frame'] >= i - tail_length) &
                (trajectories_df['frame'] <= i)
            ]

            now = window[window['frame'] == i]
            if len(now) > 0:
                coords = np.column_stack((now.x.values - x_min, now.y.values - y_min))
                dot.set_offsets(coords)
            else:
                dot.set_offsets(np.empty((0, 2)))

            for pid in particles:
                traj = window[window['track_id'] == pid].sort_values('frame')
                if len(traj) >= 2:
                    segs = [
                        [(x0 - x_min, y0 - y_min), (x1 - x_min, y1 - y_min)]
                        for (x0, y0, x1, y1) in zip(
                            traj.x.values[:-1], traj.y.values[:-1],
                            traj.x.values[1:], traj.y.values[1:]
                        )
                    ]
                    line_collections[pid].set_segments(segs)
                else:
                    line_collections[pid].set_segments([])

            return [im, dot] + list(line_collections.values())

        ani = FuncAnimation(fig, animate, frames=cropped_stack.shape[0], interval=100, blit=True)
        plt.close(fig)
        return HTML(ani.to_jshtml())

    html = animate_trajectories_cropped(tracks_in_roi, image_stack, tail_length, color)
    display(html)
    display(HTML(replace_loading_js_empty))

    print("Total number of trajectories in ROI:", len(tracks_in_roi['track_id'].unique()))


def show_tracking_compare(
    data_left, data_right,
    image_stack,
    label_left="Model A",
    label_right="Model B",
    color_left="yellow",
    color_right="cyan",
    y_min=512, y_max=768, x_min=256, x_max=512,
    tail_length=10):

    """
    Side-by-side comparison of two tracking results (animated).
    Args:
        data_left (pd.DataFrame): Tracking results for the left panel
        data_right (pd.DataFrame): Tracking results for the right panel
        image_stack (np.ndarray): Image sequence.
        label_left (str): Title of the left panel.
        label_right (str): Title of the right panel.
        color_left (str): Trajectory color (left).
        color_right (str): Trajectory color (right).
        y_min, y_max (int): Vertical ROI bounds.
        x_min, x_max (int): Horizontal ROI bounds.
        tail_length (int): Number of frames shown in trajectory tails.

    Returns:
        HTML: Synchronized side-by-side animation.
    """
    df_L = data_left.copy()
    df_R = data_right.copy()

    # Filter ROI
    def filter_roi(df):
        return df.groupby("track_id").filter(
            lambda t: (y_min < t.y.mean() < y_max) and (x_min < t.x.mean() < x_max)
        )

    df_L = filter_roi(df_L)
    df_R = filter_roi(df_R)

    cropped = image_stack[:, y_min:y_max, x_min:x_max]

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    ims = []
    dots = []
    lcs = []

    for ax, df, col, title in zip(
        axes,
        [df_L, df_R],
        [color_left, color_right],
        [label_left, label_right]
    ):
        im = ax.imshow(cropped[0], cmap="magma")
        ims.append(im)

        pids = df["track_id"].unique()
        collections = {pid: mc.LineCollection([], linewidths=1, colors=col) for pid in pids}
        for lc in collections.values():
            ax.add_collection(lc)

        dot = ax.scatter([], [], s=5, c=col)

        ax.set_title(title)
        ax.axis("off")

        dots.append(dot)
        lcs.append((collections, df))

    def animate(i):
        artists = []
        for k in range(2):
            ims[k].set_array(cropped[i])
            df = lcs[k][1]
            collections = lcs[k][0]

            window = df[(df.frame >= i - tail_length) & (df.frame <= i)]
            now = window[window.frame == i]

            if len(now):
                dots[k].set_offsets(
                    np.column_stack((now.x - x_min, now.y - y_min))
                )
            else:
                dots[k].set_offsets(np.empty((0, 2)))

            for pid, lc in collections.items():
                traj = window[window.track_id == pid].sort_values("frame")
                if len(traj) >= 2:
                    segs = [
                        [(x0 - x_min, y0 - y_min), (x1 - x_min, y1 - y_min)]
                        for x0, y0, x1, y1 in zip(
                            traj.x[:-1], traj.y[:-1],
                            traj.x[1:], traj.y[1:]
                        )
                    ]
                    lc.set_segments(segs)
                else:
                    lc.set_segments([])

            artists += [ims[k], dots[k]] + list(collections.values())

        return artists

    ani = FuncAnimation(fig, animate, frames=cropped.shape[0], interval=100, blit=True)
    plt.close(fig)

    return HTML(ani.to_jshtml())

def visualize_single_sample_comparison(model,
                                        img_tensor,
                                        gt_mask_tensor,
                                        device,
                                        threshold=0.5,
                                        sigma=1.0,
                                        model_name="Model"):
    """
    Visualizes the prediction of a single model on a specific sample.
    Args:
        model (torch.nn.Module): Trained segmentation/detection model.
        img_tensor (torch.Tensor): Input image tensor (H, W).
        gt_mask_tensor (torch.Tensor): Ground truth mask (H, W).
        device (torch.device): Device used for inference.
        threshold (float): Probability threshold for binary output.
        sigma (float): Gaussian smoothing sigma (0 disables smoothing).
        model_name (str): Label shown in the figure title.

    Shows:
        input image, ground truth mask, predicted probability map,
        and thresholded binary mask.
    """
    model.eval()

    img_tensor = img_tensor.to(device)
    gt_mask_tensor = gt_mask_tensor.to(device)

    with torch.no_grad():
        # Run through model
        logits = model(img_tensor.unsqueeze(0))
        prob_map = torch.sigmoid(logits[0, 0]).cpu().numpy()

    # Optional smoothing
    if sigma > 0:
        prob_map = gaussian_filter(prob_map, sigma=sigma)

    # Threshold for binary detection
    pred_mask = (prob_map >= threshold).astype(float)

    img = img_tensor.squeeze(0).cpu().numpy()
    gt_mask = gt_mask_tensor.squeeze(0).cpu().numpy()

    fig, axes = plt.subplots(1, 4, figsize=(16, 4))
    fig.suptitle(f"Srovnání: {model_name}", fontsize=16)

    axes[0].imshow(img, cmap='magma')
    axes[0].set_title("Input Image")
    axes[1].imshow(gt_mask, cmap='gray', vmin=0, vmax=1)
    axes[1].set_title("Ground Truth Mask")
    axes[2].imshow(prob_map, cmap='viridis')
    axes[2].set_title("Predicted Heatmap")
    axes[3].imshow(pred_mask, cmap='gray', vmin=0, vmax=1)
    axes[3].set_title(f"Thresholded Output (>{threshold})")

    for ax in axes:
        ax.axis('off')

    plt.tight_layout(rect=[0, 0.03, 1, 0.95])
    plt.show()

def show_detection_compare(
    detections_left,
    detections_right,
    image_stack,
    label_left="Detector A",
    label_right="Detector B",
    color_left="yellow",
    color_right="cyan",
    y_min=512, y_max=768, x_min=256, x_max=512,
    max_frames=None,
    interval=100
):
    """
    Side-by-side comparison of two detection results (animated).

    Args:
        detections_left (list[list[tuple]]): Per-frame detections [(x, y)] – left panel
        detections_right (list[list[tuple]]): Per-frame detections [(x, y)] – right panel
        image_stack (np.ndarray): Image sequence (frames, H, W)
        label_left (str): Title of the left panel
        label_right (str): Title of the right panel
        color_left (str): Detection color (left)
        color_right (str): Detection color (right)
        y_min, y_max, x_min, x_max (int): ROI bounds
        max_frames (int or None): Limit number of frames (default = all)
        interval (int): Animation interval in ms

    Returns:
        HTML: Side-by-side animation
    """

    def to_df(detections):
        rows = [
            (frame, x, y)
            for frame, dets in enumerate(detections)
            for (x, y) in dets
        ]
        df = pd.DataFrame(rows, columns=["frame", "x", "y"])
        return df[
            (df.y.between(y_min, y_max)) &
            (df.x.between(x_min, x_max))
        ]

    df_L = to_df(detections_left)
    df_R = to_df(detections_right)

    cropped = image_stack[:, y_min:y_max, x_min:x_max]

    n_frames = cropped.shape[0]
    if max_frames is not None:
        n_frames = min(n_frames, max_frames)

    fig, axes = plt.subplots(1, 2, figsize=(12, 5))

    ims = []
    dots = []
    dfs = [df_L, df_R]
    colors = [color_left, color_right]
    titles = [label_left, label_right]

    for ax, df, col, title in zip(axes, dfs, colors, titles):
        im = ax.imshow(cropped[0], cmap="magma")
        dot = ax.scatter([], [], s=10, c=col)

        ax.set_title(title)
        ax.axis("off")

        ims.append(im)
        dots.append(dot)

    def animate(i):
        artists = []
        for k in range(2):
            ims[k].set_array(cropped[i])
            now = dfs[k][dfs[k].frame == i]

            if len(now):
                coords = np.column_stack(
                    (now.x.values - x_min, now.y.values - y_min)
                )
                dots[k].set_offsets(coords)
            else:
                dots[k].set_offsets(np.empty((0, 2)))

            artists += [ims[k], dots[k]]

        return artists

    ani = FuncAnimation(
        fig,
        animate,
        frames=n_frames,
        interval=interval,
        blit=True
    )

    plt.close(fig)
    return HTML(ani.to_jshtml())

In [ ]:
#@title Vyhodnocení HOTA

#from baseline
def hota(gt: pd.DataFrame, tr: pd.DataFrame, threshold: float = 5) -> dict[str, float]:
    """Slightly adapted from https://github.com/JonathonLuiten/TrackEval"""

    # Ensure particle ids are sorted from 0 to max(n)
    gt = gt.copy()
    tr = tr.copy()

    gt.track_id = gt.track_id.map({old: new for old, new in zip(gt.track_id.unique(), range(gt.track_id.nunique()))})
    tr.track_id = tr.track_id.map({old: new for old, new in zip(tr.track_id.unique(), range(tr.track_id.nunique()))})

    # Initialization
    num_gt_ids = gt.track_id.nunique()
    num_tr_ids = tr.track_id.nunique()

    frames = sorted(set(gt.frame.unique()) | set(tr.frame.unique()))

    potential_matches_count = np.zeros((num_gt_ids, num_tr_ids))
    gt_id_count = np.zeros((num_gt_ids, 1))
    tracker_id_count = np.zeros((1, num_tr_ids))

    HOTA_TP, HOTA_FN, HOTA_FP = 0, 0, 0
    LocA = 0.0

    # Compute similarities (inverted normalized distance)
    similarities = [1 - np.clip(spatial.distance.cdist(gt[gt.frame == t][['x', 'y']],
                                                       tr[tr.frame == t][['x', 'y']]) / threshold, 0, 1)
                    for t in frames]

    # Accumulate global track information
    for t in frames:
        gt_ids_t = gt[gt.frame == t].track_id.to_numpy()
        tr_ids_t = tr[tr.frame == t].track_id.to_numpy()

        similarity = similarities[t]
        sim_iou_denom = similarity.sum(0)[np.newaxis, :] + similarity.sum(1)[:, np.newaxis] - similarity
        sim_iou = np.zeros_like(similarity)
        sim_iou_mask = sim_iou_denom > 0 + np.finfo('float').eps
        sim_iou[sim_iou_mask] = similarity[sim_iou_mask] / sim_iou_denom[sim_iou_mask]
        potential_matches_count[gt_ids_t[:, None], tr_ids_t[None, :]] += sim_iou

        gt_id_count[gt_ids_t] += 1
        tracker_id_count[0, tr_ids_t] += 1

    global_alignment_score = potential_matches_count / (gt_id_count + tracker_id_count - potential_matches_count)
    matches_count = np.zeros_like(potential_matches_count)

    # Calculate scores for each timestep
    for t in frames:
        gt_ids_t = gt[gt.frame == t].track_id.to_numpy()
        tr_ids_t = tr[tr.frame == t].track_id.to_numpy()

        if len(gt_ids_t) == 0:
            HOTA_FP += len(tr_ids_t)
            continue

        if len(tr_ids_t) == 0:
            HOTA_FN += len(gt_ids_t)
            continue

        similarity = similarities[t]
        score_mat = global_alignment_score[gt_ids_t[:, None], tr_ids_t[None, :]] * similarity

        match_rows, match_cols = optimize.linear_sum_assignment(-score_mat)

        actually_matched_mask = similarity[match_rows, match_cols] > 0
        alpha_match_rows = match_rows[actually_matched_mask]
        alpha_match_cols = match_cols[actually_matched_mask]

        num_matches = len(alpha_match_rows)

        HOTA_TP += num_matches
        HOTA_FN += len(gt_ids_t) - num_matches
        HOTA_FP += len(tr_ids_t) - num_matches

        if num_matches > 0:
            LocA += sum(similarity[alpha_match_rows, alpha_match_cols])
            matches_count[gt_ids_t[alpha_match_rows], tr_ids_t[alpha_match_cols]] += 1

    ass_a = matches_count / np.maximum(1, gt_id_count + tracker_id_count - matches_count)
    AssA = np.sum(matches_count * ass_a) / np.maximum(1, HOTA_TP)
    DetA = HOTA_TP / np.maximum(1, HOTA_TP + HOTA_FN + HOTA_FP)
    HOTA = np.sqrt(DetA * AssA)

    return {'HOTA': HOTA, 'AssA': AssA, 'DetA': DetA, 'LocA': LocA,
            'HOTA TP': HOTA_TP, 'HOTA FN': HOTA_FN, 'HOTA FP': HOTA_FP}


def calculate_performance(gt_path, tracks,
                          y_min=512, y_max=768, x_min=256, x_max=512,
                          name="Method"):
    """
    Calculate and print HOTA-based performance metrics for a tracking result.

    Args:
        gt_path (str): Path to ground truth CSV file.
        tracks (str | pd.DataFrame): Tracking results (as DataFrame or CSV path).
        y_min, y_max, x_min, x_max (int): ROI bounds.
        name (str): Name of the method (for display).
    """

    val_gt = pd.read_csv(gt_path)
    val_gt = val_gt.groupby('track_id').filter(
        lambda t: (y_min < t.y.mean() < y_max) and (x_min < t.x.mean() < x_max)
    )

    if isinstance(tracks, str):
        val_tracks = pd.read_csv(tracks)
    elif isinstance(tracks, pd.DataFrame):
        val_tracks = tracks.copy()
    else:
        raise TypeError("`tracks` must be a CSV path or a pandas DataFrame.")

    val_tracks = val_tracks.groupby('track_id').filter(
        lambda t: (y_min < t.y.mean() < y_max) and (x_min < t.x.mean() < x_max)
    )

    # Compute metrics
    results = hota(val_gt, val_tracks)

    print(f"{name}:")
    print(f"  HOTA: {results['HOTA']:.2f} "
          f"(AssA: {results['AssA']:.2f}, DetA: {results['DetA']:.2f})\n")

    return results

    print(f"✅ Done")

In [ ]:
#@title Pytorch Dataset

#from baseline
class SyntheticCCPDatasetTorch(torch_data.Dataset):
    """PyTorch Dataset that yields synthetic CCP images and masks. """
    # Changing these parameters too much could result in data that doesn’t match the real validation or test set.
    _min_n = 5
    _max_n = 15
    _radius = 2.5
    _contrast_fg_range = (0.0, 1.0)
    _contrast_bg_range = (0.0, 1.0)

    def __init__(self, length: int = 500):
        super().__init__()
        self.length = length
        # create one generator with the fixed parameters
        self._synthetic = SyntheticCCPDataset(
            min_n=self._min_n,
            max_n=self._max_n,
            radius=self._radius,
            contrast_fg_range=self._contrast_fg_range,
            contrast_bg_range=self._contrast_bg_range,
        )

    def __len__(self) -> int:
        return self.length

    def __getitem__(self, idx: int):
        # draw a new synthetic sample
        img, mask = self._synthetic.data_sample()

        # Normalise image to [0, 1] for better training stability
        img = (img - img.min()) / (img.max() - img.min() + 1e-6)

        # Convert numpy arrays to PyTorch tensors and add channel dimension
        img_tensor = torch.from_numpy(img).unsqueeze(0).float()
        mask_tensor = torch.from_numpy(mask).unsqueeze(0).float()

        return img_tensor, mask_tensor

In [ ]:
#@title Predikce modelů


def predict_ccps_with_model(model: nn.Module,
                            image: np.ndarray,
                            device: torch.device,
                            threshold: float = 0.2, #původně 0.5
                            sigma: float = 0.5) -> list[tuple[int, int]]: #původě 1
    """Use a trained model to detect CCPs in a single image.

    Args:
        model: trained CCPDetectorNet.
        image: 2‑D numpy array.
        device: torch device for inference.
        threshold: probability threshold for peak selection.
        sigma: optional Gaussian smoothing applied on the model output
                to reduce noise before local maxima detection.

    Returns:
        A list of (x, y) tuples indicating detected CCP positions.
    """
    model.eval()
    with torch.no_grad():
        # normalise image to [0,1]
        img_norm = (image - image.min()) / (image.max() - image.min() + 1e-6)
        tensor = torch.from_numpy(img_norm).unsqueeze(0).unsqueeze(0).float().to(device)
        logits = model(tensor)
        prob_map = torch.sigmoid(logits[0, 0]).cpu().numpy()
    # optional smoothing
    if sigma > 0:
        prob_map = gaussian_filter(prob_map, sigma=sigma)
        #prob_map = median_filter(prob_map, size = 3) #it was tried, but it gave worse results

    # threshold
    mask = prob_map >= threshold
    # local maxima in 3×3 neighbourhood
    local_max = prob_map == maximum_filter(prob_map, size=3)
    peaks = np.argwhere(local_max & mask)
    detections = [(int(col), int(row)) for row, col in peaks]
    return detections

In [ ]:
#@title Detektor
def run_detector(model, device, tif_path="val_data/val.tif",
                 threshold=0.2, sigma=0.5):
    """
    Runs the detector on the entire TIFF sequence and returns detections for each frame.

    Args:
      model: trained model (UNet, ResUNet, …)
      device: 'cpu' or 'cuda'
      tif_path: path to the TIFF containing the time sequence
      threshold: detection threshold
      sigma: smoothing sigma
    Returns:
      List[list[(x, y)]] — detections for each frame
      input_stack — the loaded TIFF for further use.
    """

    # Load val.tif
    input_stack = open_tiff_file(tif_path).astype(np.float64)

    detections_per_frame = []
    num_frames = input_stack.shape[0]

    for frame_idx in range(num_frames):
        frame = input_stack[frame_idx]
        dets = predict_ccps_with_model(
            model,
            frame,
            device,
            threshold=threshold,
            sigma=sigma
        )
        detections_per_frame.append(dets)

    print(f"✅ Detection complete. Frames processed: {num_frames}")

    return detections_per_frame, input_stack

#Tracking

In [ ]:
#@title Detektor
def run_detector(model, device, tif_path="val_data/val.tif",
                 threshold=0.2, sigma=0.5):
    """
    Runs the detector on the entire TIFF sequence and returns detections for each frame.

    Args:
      model: trained model (UNet, ResUNet, …)
      device: 'cpu' or 'cuda'
      tif_path: path to the TIFF containing the time sequence
      threshold: detection threshold
      sigma: smoothing sigma
    Returns:
      List[list[(x, y)]] — detections for each frame
      input_stack — the loaded TIFF for further use.
    """

    # Load val.tif
    input_stack = open_tiff_file(tif_path).astype(np.float64)

    detections_per_frame = []
    num_frames = input_stack.shape[0]

    for frame_idx in range(num_frames):
        frame = input_stack[frame_idx]
        dets = predict_ccps_with_model(
            model,
            frame,
            device,
            threshold=threshold,
            sigma=sigma
        )
        detections_per_frame.append(dets)

    print(f"✅ Detection complete. Frames processed: {num_frames}")

    return detections_per_frame, input_stack

In [ ]:
#@title Tracker Kalmanův filtr + Hungarian algoritmus

# Track object
class Track:
    def __init__(self, x, y, track_id):
        self.kf = KalmanFilter(dim_x=4, dim_z=2)
        self.kf.F = np.array([[1,0,1,0],
                              [0,1,0,1],
                              [0,0,1,0],
                              [0,0,0,1]])
        self.kf.H = np.array([[1,0,0,0],
                              [0,1,0,0]])
        self.kf.R *= 0.5
        self.kf.P *= 10

        # initial position
        self.kf.x[:2] = np.array([[x],[y]])

        self.id = track_id
        self.time_since_update = 0

    def predict(self):
        self.kf.predict()
        x, y = self.kf.x[0][0], self.kf.x[1][0]
        return (x, y)

    def update(self, z):
        self.kf.update(z)
        self.time_since_update = 0


# Hungarian algoritm
def hungarian_association(tracks, detections, max_dist=6):

  """
    max_dist = maximum object displacement between frames
    tracks = list of Track objects
    detections = list of (x, y) tuples
  """

  if len(tracks)==0:
        # no existing tracks → everything is new
        return [], list(range(len(detections))), []

  if len(detections)==0:
        # no detections → all tracks unmatched
        return list(range(len(tracks))), [], []

  cost = np.zeros((len(tracks), len(detections)))

    # compute distance matrix
  for i, t in enumerate(tracks):
        tx, ty = t.predict()
        for j, (x, y) in enumerate(detections):
            cost[i, j] = np.hypot(tx - x, ty - y) #eucleidian

  row_ind, col_ind = linear_sum_assignment(cost)

  matches = []
  unmatched_tracks = list(range(len(tracks)))
  unmatched_detections = list(range(len(detections)))

  for r, c in zip(row_ind, col_ind):
        if cost[r, c] < max_dist:
            matches.append((r, c))
            unmatched_tracks.remove(r)
            unmatched_detections.remove(c)

  return unmatched_tracks, unmatched_detections, matches


# tracking loop
def track_ccp_sequence(detections_per_frame, max_dist):
    tracks = []
    next_id = 0
    records = []

    for frame, detections in enumerate(detections_per_frame):

        # pre-prediction for all tracks
        _ = [t.predict() for t in tracks]

        unmatched_tracks, unmatched_dets, matches = hungarian_association(tracks, detections, max_dist)

        # update matched tracks
        for t_idx, d_idx in matches:
            x, y = detections[d_idx]
            tracks[t_idx].update(np.array([x, y]))
            records.append({
                "frame": frame,
                "x": x,
                "y": y,
                "track_id": tracks[t_idx].id
            })

        # new tracks for unmatched detections
        for d_idx in unmatched_dets:
            x, y = detections[d_idx]
            new_track = Track(x, y, next_id)
            next_id += 1
            tracks.append(new_track)
            records.append({
                "frame": frame,
                "x": x,
                "y": y,
                "track_id": new_track.id
            })

    return pd.DataFrame(records)

In [ ]:
#@title Noirfar tracker

def track_with_norfair(detections_per_frame, max_dist=8):
    """
    Track objects using the Norfair multi-object tracker..
    """
    # Initialize Norfair tracker with L1 (Manhattan) distance
    tracker = Tracker(
    distance_function="cityblock",
    distance_threshold=max_dist
)

    results = []

    for frame_idx, dets in enumerate(detections_per_frame):

        # Create Norfair detections
        norfair_dets = [
            Detection(points=np.array([[x, y]], dtype=float))
            for (x, y) in dets
        ]

        # Update tracker
        tracked = tracker.update(norfair_dets)

        # Save output
        for t in tracked:
            x, y = t.estimate[0]  # one point

            results.append({
                "frame": frame_idx,
                "x": float(x),
                "y": float(y),
                "track_id": int(t.id)
            })

    return pd.DataFrame(results)

In [ ]:
#@title Grid-search pro parametry

thresholds = [ 0.1, 0.5]# 0.2, 0.5]  # Detection probability thresholds
sigmas = [0.1, 0.5,1] # Gaussian smoothing sigmas
max_dist_values = [ 6, 8] # Max allowed displacement for tracking
input = open_tiff_file("val_data/val.tif").astype(np.float64)

def run_grid_search(model, track_method):
    """
    Run a grid search over detection and tracking parameters
    and evaluate tracking performance on validation data.

    Args:
        model (torch.nn.Module): Trained detection model.
        track_method (str): Tracking backend ("nn", "kalman", or "norfair").

    Returns:
        pd.DataFrame: Table with tracking metrics for each parameter combination.
    """

    print(f"Running grid-search with method {track_method}")
    results = []

    for md in max_dist_values:
        for th in thresholds:
            for sg in sigmas:
                print(f"Testing max_dist = {md} ...")
                print(f"Testing threshold={th}, sigma={sg} ...")

                # 1) Detection
                detections_per_frame = []
                for frame in range(len(input)):
                    dets = predict_ccps_with_model(
                        model, input[frame], device,
                        threshold=th,
                        sigma=sg
                    )
                    detections_per_frame.append(dets)

                # 2) Tracking
                if track_method == "nn":
                   tracks = link_detections(detections_per_frame, max_dist=md)
                elif track_method == "kalman":
                   tracks =track_ccp_sequence(detections_per_frame, max_dist=md)
                elif track_method == "norfair":
                   tracks = track_with_norfair(detections_per_frame, max_dist=md)

                # 3) Evaluation
                metrics = calculate_performance(
                    "val_data/val.csv",
                    tracks,
                    name="temp"
                )
                results.append({
                    "threshold": th,
                    "sigma": sg,
                    "max_dist": md,
                    "HOTA": metrics["HOTA"],
                    "AssA": metrics["AssA"],
                    "DetA": metrics["DetA"],
                })
    df = pd.DataFrame(results)
    display(df)
    return df

In [ ]:
#@title Heat map

#haet map
def plot_heatmap(df, metric="HOTA", max_dist=6, model_name="unet"):
    """
    Heatmap threshold × sigma for fixed max_dist.
    """
    df_f = df[df["max_dist"] == max_dist]

    pivot = df_f.pivot(
        index="threshold",
        columns="sigma",
        values=metric
    )

    plt.figure(figsize=(7, 5))
    sns.heatmap(pivot, annot=True, fmt=".3f", cmap="viridis")
    plt.title(f"{metric} heatmap (threshold × sigma), max_dist={max_dist}")
    plt.xlabel("Sigma")
    plt.ylabel("Threshold")

    filename = f"heatmap_{model_name}_{metric}_md{max_dist}.pdf"
    plt.savefig(filename, bbox_inches="tight")
    plt.show()

In [ ]:
#@title Nahrání natrénovaných modelů

!gdown --id 1RESdDlQCMGcceWJOE31g3wNpzKVoE4aB -O unet_trained.pth
!gdown --id 1I1tv1Qv-Z0u7lrrnLIyaS6EX5Ad0wbdE -O resunet_trained.pth

In [ ]:
#@title Načtení výsledků modelů

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model_unet = UNet()
state1 = torch.load("unet_trained.pth", map_location=device, weights_only=False)
model_unet.load_state_dict(state1)
model_unet.to(device)
model_unet.eval()

model_resunet = ResUNet()
state2 = torch.load("resunet_trained.pth", map_location=device, weights_only=False)
model_resunet.load_state_dict(state2)
model_resunet.to(device)
model_resunet.eval()

print("🔥 Models successfully loaded and ready!")

#Vyhodnocení U-Net
Parametry při trénování modelu U-net:
- Velikost datasetu 4000
- Batch size = 32
- Počet epoch = 20
- Learning rate = 0.001

In [ ]:
#@title Ladění parametrů

#run grid search and plot heat map for each max_dist
df = run_grid_search(model_unet, track_method = "nn")
for md in [ 6, 8]:
    plot_heatmap(df, metric="HOTA", max_dist=md)

# Nejlepší parametry pro U-Net
|max_dist|threshold|sigma|HOTA|tracker|
|----------------|------------|-------------|--------------|--|
|6| 0.015|0.5|0.69|NN|
|6| 0.04|0.5|0.59|Kalman|
|8| 0.04|0.5|0.48|Norfair|

In [ ]:
#@title Vyhodnocení U-Net

detections_per_frame_unet, input = run_detector(model_unet, device, threshold = 0.04, sigma = 0.5)

In [ ]:
#@title Výsledky U-Net pro Nearest-Neighbor tracker

val_tracks_nn = link_detections(detections_per_frame_unet, max_dist = 6)
results_unet1 = calculate_performance('val_data/val.csv', val_tracks_nn, name="U-net Method")

In [ ]:
#@title Výsledky U-Net pro tracker s kalmanovým filtrem

val_tracks_kalman = track_ccp_sequence(detections_per_frame_unet, max_dist = 6)
results_unet2 = calculate_performance('val_data/val.csv', val_tracks_kalman, name="U-net Method")

In [ ]:
#@title Výsledky U-Net pro Norfair tracker

val_tracks_norf = track_with_norfair(detections_per_frame_unet, max_dist = 8)
results_unet3 = calculate_performance('val_data/val.csv', val_tracks_norf, name="U-net Method")

Z uvedených výsledků lze vidět, že knihovna Norfair v kombinaci s U-Net není efektivní

In [ ]:
#@title Porovnání dvou trackingů

input_stack = open_tiff_file("val_data/val.tif").astype(np.float64)
html = show_tracking_compare(
    val_tracks_kalman,
    val_tracks_nn,
    input_stack,
    label_left="Kalman",
    label_right="NN",
    color_left="cyan",
    color_right="cyan"
)

display(html)

#Vyhodnocení Res-U-Net
Parametry při trénování modelu Res-U-net:
- Velikost datasetu 4000
- Batch size = 32
- Počet epoch = 20
- Learning rate = 0.001


In [ ]:
#@title Ladění parametrů Res-U-Net

df_resunet = run_grid_search(model_resunet, track_method = "nn")
for md in [ 6, 8]:
    plot_heatmap(df_resunet, metric="HOTA", max_dist=md)

# Nejlepší parametry pro Res-U-Net
| max_dist|threshold|sigma|HOTA|tracker|
|----------------|------------|-------------|--------------|--|
|6|0.015|0.5|0.72|NN|
|6|0.05|0.5|0.72|NN|
|8|0.015|0.5|0.71|NN|

In [ ]:
#@title Vyhodnocení Res-U-Net

detections_per_frame_resunet, input = run_detector(model_resunet, device, threshold = 0.05, sigma = 1)

In [ ]:
#@title Vyhodnocení Res-U-Net pro NN tracker

val_tracks_nn_res = link_detections(detections_per_frame_resunet, max_dist = 6)
results_resunet1 = calculate_performance('val_data/val.csv', val_tracks_nn_res, name="Res-U-net Method")

In [ ]:
#@title Vyhodnocení Res-U-Net pro tracker s kalmanovým filtrem

val_tracks_kalman_res = track_ccp_sequence(detections_per_frame_resunet, max_dist = 6)
results_resunet2 = calculate_performance('val_data/val.csv', val_tracks_kalman_res, name="Res-U-net Method")

In [ ]:
#@title Vyhodnocení Res-U-Net pro Norfair tracker

val_tracks_norf_res = track_with_norfair(detections_per_frame_resunet, max_dist = 5)
results_resunet3 = calculate_performance('val_data/val.csv', val_tracks_norf_res, name="Res-U-net Method")

In [ ]:
#@title Porovnání dvou trackingů

input_stack = open_tiff_file("val_data/val.tif").astype(np.float64)
html = show_tracking_compare(
    val_tracks_kalman_res,
    val_tracks_nn_res,
    input_stack,
    label_left="Kalman",
    label_right="NN",
    color_left="cyan",
    color_right="cyan"
)

display(html)

In [ ]:
#@title Porovnání dvou modelů na syntetickém datasetu

img, gt_mask = next(iter(synthetic_dataset))
visualize_single_sample_comparison(
    model_unet, img,gt_mask,
    device, threshold = 0.1, sigma = 0.5,
    model_name="U-Net")

visualize_single_sample_comparison(
    model_resunet,img,gt_mask,
    device, threshold = 0.1, sigma = 0.5,
    model_name="Res-U-Net")

In [ ]:
#@title Vizualizace detekce na obou modelech

input_stack = open_tiff_file("val_data/val.tif").astype(np.float64)
html = show_detection_compare(
    detections_left=detections_per_frame_unet,
    detections_right=detections_per_frame_resunet,
    image_stack=input_stack,
    label_left="U-Net",
    label_right="Res-U-Net",
    color_left="cyan",
    color_right="cyan")

display(html)

In [ ]:
#@title SOTA
results_sota = calculate_performance('val_data/val.csv', 'val_data/sota.csv', name="SOTA Method")

In [ ]:
#@title Tabulka výsledků

summary_data = {
    "Method": ["U-Net", "Res-U-Net", "SOTA Method"],
    "HOTA": [results_unet1["HOTA"], results_resunet1["HOTA"], results_sota["HOTA"]],
    "AssA": [results_unet1["AssA"], results_resunet1["AssA"], results_sota["AssA"]],
    "DetA": [results_unet1["DetA"], results_resunet1["DetA"], results_sota["DetA"]],
}

summary_df = pd.DataFrame(summary_data)

# Round to two decimals for readability
summary_df = summary_df.round(2)
display(summary_df.style.set_properties(**{
    'text-align': 'center',
    'font-weight': 'bold'
}).set_table_styles([{
    'selector': 'th',
    'props': [('text-align', 'center'), ('font-weight', 'bold')]
}]))

In [ ]:
#@title Porovnání výsledků metod

methods = ["Morphological", "CNN baseline", "U-Net", "ResUNet", "SOTA"]
HOTA  = [0.23, 0.31, results_unet1["HOTA"], results_resunet1["HOTA"], results_sota["HOTA"]]
AssA  = [0.23, 0.27, results_unet1["AssA"], results_resunet1["AssA"], results_sota["AssA"]]
DetA  = [0.24, 0.36, results_unet1["DetA"], results_resunet1["DetA"], results_sota["DetA"]]

x = np.arange(len(methods))
width = 0.25

plt.figure(figsize=(12,6))
plt.bar(x - width, HOTA,  width, label="HOTA")
plt.bar(x,          AssA, width, label="AssA")
plt.bar(x + width,  DetA, width, label="DetA")

plt.xticks(x, methods, rotation=30)
plt.ylabel("Score")
plt.title("Comparison of tracking performance across methods")
plt.legend()
plt.grid(axis="y")
plt.tight_layout()
plt.show()

In [ ]:
#@title Výpočet precission a recall

def match_points(gt_points, pred_points, max_dist=5):
    if len(gt_points) == 0 and len(pred_points) == 0:
        return 0, 0, 0

    if len(gt_points) == 0:
        return 0, len(pred_points), 0

    if len(pred_points) == 0:
        return 0, 0, len(gt_points)

    dists = spatial.distance.cdist(gt_points, pred_points)

    # Hungarian matching
    gt_idx, pred_idx = linear_sum_assignment(dists)

    # keep only matches within max_dist
    valid = dists[gt_idx, pred_idx] <= max_dist

    tp = np.sum(valid)
    fp = len(pred_points) - tp
    fn = len(gt_points) - tp

    return tp, fp, fn

def compute_precision_recall(gt_df, detections_per_frame, max_dist=5):
    TP = FP = FN = 0

    for frame in range(len(detections_per_frame)):
        gt_f = gt_df[gt_df.frame == frame]
        gt_points = gt_f[["x", "y"]].values
        pred_points = np.array(detections_per_frame[frame])

        tp, fp, fn = match_points(gt_points, pred_points, max_dist)

        TP += tp
        FP += fp
        FN += fn

    precision = TP / (TP + FP + 1e-9)
    recall = TP / (TP + FN + 1e-9)
    f1 = 2 * precision * recall / (precision + recall + 1e-9)

    return {
        "Precision": precision,
        "Recall": recall,
        "F1": f1,
        "TP": TP,
        "FP": FP,
        "FN": FN
    }

def tracks_to_detections_per_frame(tracks_df, num_frames):
    detections = [[] for _ in range(num_frames)]

    for _, row in tracks_df.iterrows():
        f = int(row.frame)
        detections[f].append((row.x, row.y))

    return detections

gt = pd.read_csv("val_data/val.csv")
sota_tracks = pd.read_csv("val_data/sota.csv")

detections_sota = tracks_to_detections_per_frame(
    sota_tracks,
    num_frames=len(input)
)


In [ ]:
#@title Precission a recall porovnání

gt = pd.read_csv("val_data/val.csv")

pr_unet = compute_precision_recall(gt, detections_per_frame_unet, max_dist=6)
pr_resunet = compute_precision_recall(gt, detections_per_frame_resunet, max_dist=6)
pr_sota = compute_precision_recall(gt, detections_sota, max_dist=6)

pd.DataFrame([pr_sota, pr_unet, pr_resunet],
             index=["SOTA", "U-Net", "ResUNet"])

In [ ]:
#@title Nejlepší model

model_best = model_resunet
detections_per_frame_best, input = run_detector(model_best, device, threshold = 0.05, sigma = 0.5)
val_tracks = link_detections(detections_per_frame_best, max_dist = 6)
best_results = calculate_performance('val_data/val.csv', val_tracks, name="Best Method")

input_stack = open_tiff_file("val_data/val.tif").astype(np.float64)
show_tracking(val_tracks, input_stack )

# Závěr
Byla analyzována data buněk CCP pomocí několika metod strojového učení s cílem přiblížit se výsledkům referenční metody SOTA nebo je překonat. Za nejúspěšnější metodu je dle výsledků považována metoda ResUNet spolu s NN trackingem, která se liší od SOTA výsledků v rámci HOTA pouze o 0.02. Přestože nebyla překonána referenční metoda SOTA, byly nalezeny metody a parametry, které se k jejím hodnotám v rámci metrik HOTA, AssA a DetA velice blíží

Pro lepší výsledky by se nabízely následující směry:
Použití pokročilejších modelů (např. UNet++ nebo menší ViT-based architektury).
Zkusit jinou loss funkci, například kombinace BCE + Dice.

Pokročilejší tracking, např. DeepSORT, multi-hypothesis tracking nebo probabilistic data association.


#Reference
 Prince, S. J. D. (2023). *Understanding Deep Learning*. MIT Press

 (https://github.com/tryolabs/norfair)

